In [1]:
import os 
import pandas as pd
from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [36]:
PATH_CSV_LABELS = 'tabellaPseudoLabelsDatasetCompleto.csv'
PATH_DATASET_LAVORATO = '../Nuovi Dati/TemplateGeneratiClean'

PATH_MODELLO_SPECKLE = '../Modelli-v2/modello_finale_riaddestrato_speckle'
PATH_MODELLO_GAUSSIAN_BLUR = '../Modelli-v2/modello_finale_riaddestrato_gaussian_blur'
PATH_MODELLO_BLACK_STAINS = '../Modelli-v2/modello_finale_riaddestrato_black_stains'
# PATH_MODELLO_BLACK_STAINS = '../Modelli-v2/modello_pesi_inizializzati_black_stains'

N_IMG = 1
FORMATO_IMG = 'jpg'
CHANNELS = 1
HEIGHT = 512
WIDTH = 512
TARGET_SIZE = (WIDTH, HEIGHT)

In [3]:
df_labels = pd.read_csv(PATH_CSV_LABELS)
df_labels.head(5)

,Acquisizione,Label,Label_norm
0,AgatielloRoberto_000,0.648876,50.428711
1,AgatielloRoberto_001,0.759503,61.300294
2,AgatielloRoberto_002,0.747500,60.120716
3,AgatielloRoberto_003,0.818899,67.137182
4,AgatielloRoberto_004,0.825895,67.824714


In [45]:
X = []
y = []

for id_acquisizione in df_labels.Acquisizione:
    path_img = os.path.join(PATH_DATASET_LAVORATO, id_acquisizione, 'Immagini', 'immagine1.jpg')
    # print(path_img)
    # path_img = os.path.join('../Nuovi Dati/DatasetDistorti/DatasetBlackStains-t3', id_acquisizione, 'Immagini', 'immagine1.jpg')
    image = Image.open(path_img)
    resized_image = image.resize(TARGET_SIZE)
    image_array = np.array(resized_image)
    normalized_image_array = image_array / 255.0
    
    pseudo_label = df_labels[df_labels.Acquisizione == id_acquisizione].Label_norm
    
    X.append(normalized_image_array)
    y.append(pseudo_label)
    
X = np.array(X)
y = np.array(y)

In [42]:
# test_set = df_labels.Acquisizione.apply(lambda x:  x.startswith('TozziVito'))
# test_set = df_labels[test_set]

In [43]:
# X = []
# y = []

# for id_acquisizione in test_set.Acquisizione.unique():
#     print(path_img)
#     path_img = os.path.join(PATH_DATASET_LAVORATO, id_acquisizione, 'Immagini', 'immagine1.jpg')
#     image = Image.open(path_img)
#     resized_image = image.resize(TARGET_SIZE)
#     image_array = np.array(resized_image)
#     normalized_image_array = image_array / 255.0
    
#     pseudo_label = df_labels[df_labels.Acquisizione == id_acquisizione].Label_norm
    
#     X.append(normalized_image_array)
#     y.append(pseudo_label)
    
# X = np.array(X)
# y = np.array(y)

../Nuovi Dati/TemplateGeneratiClean/VenafraSara_014/Immagini/immagine1.jpg
../Nuovi Dati/TemplateGeneratiClean/TozziVito_000/Immagini/immagine1.jpg
../Nuovi Dati/TemplateGeneratiClean/TozziVito_001/Immagini/immagine1.jpg
../Nuovi Dati/TemplateGeneratiClean/TozziVito_002/Immagini/immagine1.jpg
../Nuovi Dati/TemplateGeneratiClean/TozziVito_003/Immagini/immagine1.jpg


In [37]:
model_speckle = tf.keras.models.load_model(PATH_MODELLO_SPECKLE)
model_gaussian_blur = tf.keras.models.load_model(PATH_MODELLO_GAUSSIAN_BLUR)
model_black_stains = tf.keras.models.load_model(PATH_MODELLO_BLACK_STAINS)

classificatore_blur = tf.keras.models.load_model('/Users/simone/Documents/Universita/Sensori/IQA-PROGETTO-GIT/Utilità python/classificatori/modelli_classificatori/classificatore_blur')
classificatore_black_stains = tf.keras.models.load_model('/Users/simone/Documents/Universita/Sensori/IQA-PROGETTO-GIT/Utilità python/classificatori/modelli_classificatori/classificatore_black_stains')
classificatore_polvere = tf.keras.models.load_model('/Users/simone/Documents/Universita/Sensori/IQA-PROGETTO-GIT/Utilità python/classificatori/modelli_classificatori/classificatore_polvere')

In [7]:
previsioni_nuovo_modello_speckle = model_speckle.predict(X)
previsioni_nuovo_modello_gaussian_blur = model_gaussian_blur.predict(X)
previsioni_nuovo_modello_black_stains = model_black_stains.predict(X)

2023-07-26 15:52:11.349949: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-07-26 15:52:11.407859: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


 1/23 [>.............................] - ETA: 2s

2023-07-26 15:52:15.709059: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


 1/23 [>.............................] - ETA: 1s

2023-07-26 15:52:20.956941: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


23/23 [==============================] - 4s 192ms/step


In [26]:
previsioni_classificatore_gaussian_blur = classificatore_blur.predict(X)
previsioni_classificatore_polvere = classificatore_polvere.predict(X)
previsioni_classificatore_black_stains = classificatore_black_stains.predict(X)

 1/23 [>.............................] - ETA: 3s

2023-07-26 16:02:10.560225: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


 1/23 [>.............................] - ETA: 2s

2023-07-26 16:02:17.489724: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


23/23 [==============================] - 7s 309ms/step


In [35]:
len(previsioni_classificatore_gaussian_blur)

718

In [30]:
previsioni_list_blur = [i[0] for i in previsioni_classificatore_gaussian_blur]
a = sum([0 if i < 0.99 else 1 for i in previsioni_list_blur])
bool_val_blur = [False if i < 0.99 else True for i in previsioni_list_blur]
a

218

In [27]:
# previsioni_list_blur = [i[0] for i in previsioni_classificatore_polvere]
# a = sum([0 if i < 0.99 else 1 for i in previsioni_list_blur])
# a

572

In [32]:
previsioni_list_black_stains = [i[0] for i in previsioni_classificatore_black_stains]
a = sum([0 if i < 0.99 else 1 for i in previsioni_list_black_stains])
bool_val_black_stains = [False if i < 0.99 else True for i in previsioni_list_black_stains]
a

161

In [110]:
y

array([[29.0905992 ],
       [13.11175912],
       [ 7.7756871 ],
       [16.64850159],
       [16.21545199]])

In [112]:
list(previsioni_nuovo_modello)

[array([96.51858], dtype=float32),
 array([102.70201], dtype=float32),
 array([-0.00116403], dtype=float32),
 array([24.030027], dtype=float32),
 array([27.18772], dtype=float32)]

In [57]:
bool_nn_black_speckle = [True if i < 20 else False for i in previsioni_nuovo_modello_speckle]
bool_nn_black_blur = [True if i < 20 else False for i in previsioni_nuovo_modello_gaussian_blur]
bool_nn_black_stains = [True if i < 20 else False for i in previsioni_nuovo_modello_black_stains]

In [58]:
a = df_labels.copy()
a['class_back_stains'] = bool_val_black_stains
a['class_blur'] = bool_val_blur
a['bool_nn_black_speckle'] = bool_nn_black_speckle
a['bool_nn_black_blur'] = bool_nn_black_blur
a['bool_nn_black_stains'] = bool_nn_black_stains
a

,Acquisizione,Label,Label_norm,class_back_stains,class_blur,bool_nn_black_speckle,bool_nn_black_blur,bool_nn_black_stains
0,AgatielloRoberto_000,0.648876,50.428711,False,False,False,False,False
1,AgatielloRoberto_001,0.759503,61.300294,False,False,False,False,False
2,AgatielloRoberto_002,0.747500,60.120716,False,False,False,False,False
3,AgatielloRoberto_003,0.818899,67.137182,False,False,False,False,False
4,AgatielloRoberto_004,0.825895,67.824714,False,False,False,False,False
...,...,...,...,...,...,...,...,...
713,VenafraSara_010,0.638803,49.438784,False,False,False,False,False
714,VenafraSara_011,0.789886,64.286081,False,False,False,False,False
715,VenafraSara_012,0.789531,64.251135,False,False,False,False,False
716,VenafraSara_013,0.766965,62.033559,False,False,False,False,False


In [63]:
acquisizioni_filtrate = a[(~a['class_back_stains']) & (~a['class_blur']) & (~a['bool_nn_black_speckle']) & (~a['bool_nn_black_blur']) & (~a['bool_nn_black_stains']) ]
    

In [65]:
lista_acquisizioni_filtrate = acquisizioni_filtrate.Acquisizione.unique()

In [67]:
len(lista_acquisizioni_filtrate

418

In [75]:
# lista_acquisizioni = list(df_labels[previsioni_nuovo_modello < 35].Acquisizione.unique())
# PATH_FILE_SCORE = '../scoreCsvDatasetFinale.csv'
# scores = pd.read_csv(PATH_FILE_SCORE)
# scores = scores[~scores.Utente1.isin(lista_acquisizioni)]
# scores = scores[~scores.Utente2.isin(lista_acquisizioni)]
# scores

In [69]:
lista_acquisizioni = list(set(df_labels.Acquisizione.unique()) - set(lista_acquisizioni_filtrate))

PATH_FILE_SCORE = '../scoreCsvDatasetFinale.csv'
scores = pd.read_csv(PATH_FILE_SCORE)
scores = scores[~scores.Utente1.isin(lista_acquisizioni)]
scores = scores[~scores.Utente2.isin(lista_acquisizioni)]
scores

,Utente1,Utente2,ScoreML
0,AgatielloRoberto_000,AgatielloRoberto_001,0.526569
1,AgatielloRoberto_000,AgatielloRoberto_002,0.491637
2,AgatielloRoberto_000,AgatielloRoberto_003,0.525314
3,AgatielloRoberto_000,AgatielloRoberto_004,0.516008
4,AgatielloRoberto_000,AgatielloRoberto_005,0.567200
...,...,...,...
258116,VenafraSara_012,VenafraSara_014,0.593949
258117,VenafraSara_012,VenturaAlessio_000,0.121388
258118,VenafraSara_013,VenafraSara_014,0.599829
258119,VenafraSara_013,VenturaAlessio_000,0.121099


In [70]:
# scores.to_csv('../scoreFiltratiFinaliCsv.csv' ,index=False)
scores.to_csv('../scoreFiltratiFinaliCsvSuperFiltraggio.csv' ,index=False)